<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/notebooks/InraTour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import itertools
import math
import plotly.express as px
import pandas as pd

In [3]:
home = ['Pleione', (-77 , -146.78125 , -344.125)]

In [18]:
points = [    ['12 Trianguli',  (-77.15625 , -76.8125 , -120.6875)],
              ['HIP 12099',     (-101.90625 , -95.46875 , -165.59375)],
              ['HIP 16824',     (-49.21875 , -89.375 , -187.1875)],
              ['HIP 15329',     (-87.84375 , -52 , -167.375)],
              ['Alnath',        (-3.875 , -7.375 , -133.65625)],
              ['HIP 7158',      (-3.78125 , -143.84375 , -33.8125)],
              ['HIP 59382',     (19.03125 , 130.09375 , -11.59375)],
              ['Conn',          (-36.59375 , -9.1875 , -20.5)],
              ['Hermitage',     (-28.75 , 25 , 10.4375)],
              ['LP 389-95',     (98.84375 , 7 , -7.71875)],
          ]

In [5]:
sequence = list(range( 0,len(points)))

In [11]:

def dist( pt1, pt2):
  d = 0.0
  d += (pt1[0] - pt2[0]) **2
  d += (pt1[1] - pt2[1]) **2
  d += (pt1[2] - pt2[2]) **2
  return math.sqrt(d)

def score(sequence):
  d = 0.0
  slen = len(sequence)
  d+=  dist( home[1], points[sequence[0]][1])
  for i in range(1, slen):
    d+= dist( points[sequence[i]][1], points[sequence[i-1]][1])

  d+= dist( home[1], points[sequence[slen-1]][1])
  return d

def search_for_best():
  i = 0
  magnitude = 2

  best = sequence
  bestScore = score(sequence)
  for cur in itertools.permutations(sequence):
   if i % 10 **magnitude == 0:
    if i % 10 **(magnitude+2) == 0:
      print(".")
    else:
      print(".", end='')

    sco = score(cur)
    if sco< bestScore:
      best = cur
      bestScore = sco
   i = i + 1

  print("")
  print(str(bestScore))
  print(best)

1334.5975515058062
(4, 9, 6, 8, 7, 5, 0, 3, 1, 2)

In [12]:
def nicePrint( seq ):
  d = dist( home[1], points[seq[0]][1])
  print(f"{home[0]}\t\t {d:.0f}")
  for i in range(0, len(seq)-1):
    d= dist( points[seq[i]][1], points[seq[i+1]][1])
    print(f"{points[i][0]}\t\t {d:.0f}")
  
  d= dist( home[1], points[seq[-1]][1])
  print(f"{points[seq[-1]][0]}\t\t {d:.0f}")
  print(f"{home[0]}\t\t -")


In [13]:
nicePrint( (4, 9, 6, 8, 7, 5, 0, 3, 1, 2) )

Pleione		 263
12 Trianguli		 163
HIP 12099		 147
HIP 16824		 118
HIP 15329		 47
Alnath		 139
HIP 7158		 132
HIP 59382		 54
Conn		 46
Hermitage		 57
HIP 16824		 169
Pleione		 -


In [22]:
def makeFrame( seq ):
  names = []
  xs = []
  ys = []
  zs = []

  names.append( home[0])
  xs.append( home[1][0])
  ys.append( home[1][1])
  zs.append( home[1][2])


  for i in range(0, len(seq)):
    thePoint = points[seq[i]]
    names.append( thePoint[0])
    xs.append( thePoint[1][0])
    ys.append( thePoint[1][1])
    zs.append( thePoint[1][2])

  names.append( home[0])
  xs.append( home[1][0])
  ys.append( home[1][1])
  zs.append( home[1][2])

  data = { 'names': names, 'xs':xs, 'ys':ys, 'zs':zs }
  return pd.DataFrame( data)

In [23]:
df = makeFrame( (4, 9, 6, 8, 7, 5, 0, 3, 1, 2) )

In [33]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter3d(
    x=df['xs'],
    y=df['ys'],
    z=df['zs'],
    mode='markers+lines',
)])

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()